# Structured Outputs For Reasoning Models

When working with reasoning models that use special tokens like `<think>...</think>` to denote reasoning sections, you might want to allow free-form text within these sections while still enforcing grammar constraints on the rest of the output.

SGLang provides a feature to disable grammar restrictions within reasoning sections. This is particularly useful for models that need to perform complex reasoning steps before providing a structured output.

To enable this feature, use the `--reasoning-parser` flag which decide the think_end_token, such as `</think>`, when launching the server. You can also specify the reasoning parser using the `--reasoning-parser` flag.

## Supported Models

Currently, SGLang supports the following reasoning models:
- [DeepSeek R1 series](https://huggingface.co/collections/deepseek-ai/deepseek-r1-678e1e131c0169c0bc89728d): The reasoning content is wrapped with `<think>` and `</think>` tags.
- [QwQ](https://huggingface.co/Qwen/QwQ-32B): The reasoning content is wrapped with `<think>` and `</think>` tags.


## Usage

## OpenAI Compatible API

Specify the `--grammar-backend`, `--reasoning-parser` option.

In [1]:
import openai
import os

from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

os.environ["TOKENIZERS_PARALLELISM"] = "false"


server_process, port = launch_server_cmd(
    "python -m sglang.launch_server --model-path deepseek-ai/DeepSeek-R1-Distill-Qwen-7B --host 0.0.0.0 --reasoning-parser deepseek-r1"
)

wait_for_server(f"http://localhost:{port}")
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

W0822 15:29:25.367000 4046732 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 15:29:25.367000 4046732 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 15:29:26] server_args=ServerArgs(model_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_path='deepseek-ai/DeepSeek-R1-Distill-Qwen-7B', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=39046, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.874, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False,

[2025-08-22 15:29:26] Using default HuggingFace chat template with detected content format: string


W0822 15:29:34.599000 4046981 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 15:29:34.599000 4046981 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 15:29:34.650000 4046980 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 15:29:34.650000 4046980 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 15:29:35] Attention backend not explicitly specified. Use fa3 backend by default.
[2025-08-22 15:29:35] Init torch distributed begin.


...

[2025-08-22 15:29:44] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=37.09 GB


[2025-08-22 15:29:44] INFO:     Started server process [4046732]
[2025-08-22 15:29:44] INFO:     Waiting for application startup.
[2025-08-22 15:29:44] INFO:     Application startup complete.
[2025-08-22 15:29:44] INFO:     Uvicorn running on http://0.0.0.0:39046 (Press CTRL+C to quit)


[2025-08-22 15:29:45] INFO:     127.0.0.1:45184 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 15:29:45] INFO:     127.0.0.1:45194 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 15:29:45] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:29:46] INFO:     127.0.0.1:45204 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 15:29:46] The server is fired up and ready to roll!


### JSON

you can directly define a JSON schema or use [Pydantic](https://docs.pydantic.dev/latest/) to define and validate the response.

**Using Pydantic**

In [2]:
from pydantic import BaseModel, Field


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "foo",
            # convert the pydantic model to json schema
            "schema": CapitalInfo.model_json_schema(),
        },
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-22 15:29:50] Prefill batch. #new-seq: 1, #new-token: 21, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:29:51] Decode batch. #running-req: 1, #token: 55, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.28, #queue-req: 0, 


[2025-08-22 15:29:52] Decode batch. #running-req: 1, #token: 95, token usage: 0.00, cuda graph: True, gen throughput (token/s): 158.39, #queue-req: 0, 


[2025-08-22 15:29:52] Decode batch. #running-req: 1, #token: 135, token usage: 0.01, cuda graph: True, gen throughput (token/s): 169.49, #queue-req: 0, 


[2025-08-22 15:29:52] Decode batch. #running-req: 1, #token: 175, token usage: 0.01, cuda graph: True, gen throughput (token/s): 150.35, #queue-req: 0, 


...

[2025-08-22 15:29:53] Decode batch. #running-req: 1, #token: 295, token usage: 0.01, cuda graph: True, gen throughput (token/s): 156.53, #queue-req: 0, 


[2025-08-22 15:29:53] Decode batch. #running-req: 1, #token: 335, token usage: 0.02, cuda graph: True, gen throughput (token/s): 169.01, #queue-req: 0, 


[2025-08-22 15:29:53] Decode batch. #running-req: 1, #token: 375, token usage: 0.02, cuda graph: True, gen throughput (token/s): 158.63, #queue-req: 0, 


[2025-08-22 15:29:54] Decode batch. #running-req: 1, #token: 415, token usage: 0.02, cuda graph: True, gen throughput (token/s): 124.18, #queue-req: 0, 


[2025-08-22 15:29:54] Decode batch. #running-req: 1, #token: 455, token usage: 0.02, cuda graph: True, gen throughput (token/s): 104.82, #queue-req: 0, 
[2025-08-22 15:29:54] INFO:     127.0.0.1:45220 - "POST /v1/chat/completions HTTP/1.1" 200 OK


**JSON Schema Directly**


In [3]:
import json

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    response_format={
        "type": "json_schema",
        "json_schema": {"name": "foo", "schema": json.loads(json_schema)},
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-22 15:29:54] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 21, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:29:55] Decode batch. #running-req: 1, #token: 56, token usage: 0.00, cuda graph: True, gen throughput (token/s): 90.50, #queue-req: 0, 


[2025-08-22 15:29:55] Decode batch. #running-req: 1, #token: 96, token usage: 0.00, cuda graph: True, gen throughput (token/s): 148.55, #queue-req: 0, 


[2025-08-22 15:29:55] Decode batch. #running-req: 1, #token: 136, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.52, #queue-req: 0, 


[2025-08-22 15:29:55] Decode batch. #running-req: 1, #token: 176, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.84, #queue-req: 0, 


...

[2025-08-22 15:29:56] Decode batch. #running-req: 1, #token: 296, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.45, #queue-req: 0, 


[2025-08-22 15:29:56] Decode batch. #running-req: 1, #token: 336, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.47, #queue-req: 0, 


[2025-08-22 15:29:56] Decode batch. #running-req: 1, #token: 376, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.45, #queue-req: 0, 


[2025-08-22 15:29:57] Decode batch. #running-req: 1, #token: 416, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.30, #queue-req: 0, 


[2025-08-22 15:29:57] Decode batch. #running-req: 1, #token: 456, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.20, #queue-req: 0, 
[2025-08-22 15:29:57] INFO:     127.0.0.1:45220 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### EBNF

In [4]:
ebnf_grammar = """
root ::= city | description
city ::= "London" | "Paris" | "Berlin" | "Rome"
description ::= city " is " status
status ::= "the capital of " country
country ::= "England" | "France" | "Germany" | "Italy"
"""

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "system", "content": "You are a helpful geography bot."},
        {
            "role": "assistant",
            "content": "Give me the information and population of the capital of France in the JSON format.",
        },
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"ebnf": ebnf_grammar},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-22 15:29:57] Prefill batch. #new-seq: 1, #new-token: 28, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:29:57] Decode batch. #running-req: 1, #token: 64, token usage: 0.00, cuda graph: True, gen throughput (token/s): 148.18, #queue-req: 0, 


[2025-08-22 15:29:57] Decode batch. #running-req: 1, #token: 104, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.55, #queue-req: 0, 


[2025-08-22 15:29:58] Decode batch. #running-req: 1, #token: 144, token usage: 0.01, cuda graph: True, gen throughput (token/s): 166.88, #queue-req: 0, 


[2025-08-22 15:29:58] Decode batch. #running-req: 1, #token: 184, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.80, #queue-req: 0, 


...

[2025-08-22 15:30:00] Decode batch. #running-req: 1, #token: 584, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.06, #queue-req: 0, 


[2025-08-22 15:30:01] Decode batch. #running-req: 1, #token: 624, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.08, #queue-req: 0, 


[2025-08-22 15:30:01] Decode batch. #running-req: 1, #token: 664, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.04, #queue-req: 0, 


[2025-08-22 15:30:01] Decode batch. #running-req: 1, #token: 704, token usage: 0.03, cuda graph: True, gen throughput (token/s): 165.01, #queue-req: 0, 


[2025-08-22 15:30:01] Decode batch. #running-req: 1, #token: 744, token usage: 0.04, cuda graph: True, gen throughput (token/s): 165.02, #queue-req: 0, 
[2025-08-22 15:30:01] INFO:     127.0.0.1:45220 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Regular expression

In [5]:
response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=[
        {"role": "assistant", "content": "What is the capital of France?"},
    ],
    temperature=0,
    max_tokens=2048,
    extra_body={"regex": "(Paris|London)"},
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-22 15:30:01] Prefill batch. #new-seq: 1, #new-token: 11, #cached-token: 2, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:30:02] Decode batch. #running-req: 1, #token: 34, token usage: 0.00, cuda graph: True, gen throughput (token/s): 149.57, #queue-req: 0, 


[2025-08-22 15:30:02] Decode batch. #running-req: 1, #token: 74, token usage: 0.00, cuda graph: True, gen throughput (token/s): 167.60, #queue-req: 0, 


[2025-08-22 15:30:02] Decode batch. #running-req: 1, #token: 114, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.52, #queue-req: 0, 
[2025-08-22 15:30:02] INFO:     127.0.0.1:45220 - "POST /v1/chat/completions HTTP/1.1" 200 OK


### Structural Tag

In [6]:
tool_get_current_weather = {
    "type": "function",
    "function": {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {
                    "type": "string",
                    "description": "The city to find the weather for, e.g. 'San Francisco'",
                },
                "state": {
                    "type": "string",
                    "description": "the two-letter abbreviation for the state that the city is"
                    " in, e.g. 'CA' which would mean 'California'",
                },
                "unit": {
                    "type": "string",
                    "description": "The unit to fetch the temperature in",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["city", "state", "unit"],
        },
    },
}

tool_get_current_date = {
    "type": "function",
    "function": {
        "name": "get_current_date",
        "description": "Get the current date and time for a given timezone",
        "parameters": {
            "type": "object",
            "properties": {
                "timezone": {
                    "type": "string",
                    "description": "The timezone to fetch the current date and time for, e.g. 'America/New_York'",
                }
            },
            "required": ["timezone"],
        },
    },
}

schema_get_current_weather = tool_get_current_weather["function"]["parameters"]
schema_get_current_date = tool_get_current_date["function"]["parameters"]


def get_messages():
    return [
        {
            "role": "system",
            "content": f"""
# Tool Instructions
- Always execute python code in messages that you share.
- When looking for real time information use relevant functions if available else fallback to brave_search
You have access to the following functions:
Use the function 'get_current_weather' to: Get the current weather in a given location
{tool_get_current_weather["function"]}
Use the function 'get_current_date' to: Get the current date and time for a given timezone
{tool_get_current_date["function"]}
If a you choose to call a function ONLY reply in the following format:
<{{start_tag}}={{function_name}}>{{parameters}}{{end_tag}}
where
start_tag => `<function`
parameters => a JSON dict with the function argument name as key and function argument value as value.
end_tag => `</function>`
Here is an example,
<function=example_function_name>{{"example_name": "example_value"}}</function>
Reminder:
- Function calls MUST follow the specified format
- Required parameters MUST be specified
- Only call one function at a time
- Put the entire function call reply on one line
- Always add your sources when using search results to answer the user query
You are a helpful assistant.""",
        },
        {
            "role": "assistant",
            "content": "You are in New York. Please get the current date and time, and the weather.",
        },
    ]


messages = get_messages()

response = client.chat.completions.create(
    model="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    messages=messages,
    response_format={
        "type": "structural_tag",
        "max_new_tokens": 2048,
        "structures": [
            {
                "begin": "<function=get_current_weather>",
                "schema": schema_get_current_weather,
                "end": "</function>",
            },
            {
                "begin": "<function=get_current_date>",
                "schema": schema_get_current_date,
                "end": "</function>",
            },
        ],
        "triggers": ["<function="],
    },
)

print_highlight(
    f"reasoing_content: {response.choices[0].message.reasoning_content}\n\ncontent: {response.choices[0].message.content}"
)

[2025-08-22 15:30:02] Prefill batch. #new-seq: 1, #new-token: 472, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:30:02] Decode batch. #running-req: 1, #token: 494, token usage: 0.02, cuda graph: True, gen throughput (token/s): 113.94, #queue-req: 0, 


[2025-08-22 15:30:03] Decode batch. #running-req: 1, #token: 534, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.67, #queue-req: 0, 


[2025-08-22 15:30:03] Decode batch. #running-req: 1, #token: 574, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.89, #queue-req: 0, 


[2025-08-22 15:30:03] Decode batch. #running-req: 1, #token: 614, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.83, #queue-req: 0, 


...

[2025-08-22 15:30:04] Decode batch. #running-req: 1, #token: 694, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.69, #queue-req: 0, 


[2025-08-22 15:30:04] Decode batch. #running-req: 1, #token: 734, token usage: 0.04, cuda graph: True, gen throughput (token/s): 162.62, #queue-req: 0, 


[2025-08-22 15:30:04] Decode batch. #running-req: 1, #token: 774, token usage: 0.04, cuda graph: True, gen throughput (token/s): 162.66, #queue-req: 0, 


[2025-08-22 15:30:04] Decode batch. #running-req: 1, #token: 814, token usage: 0.04, cuda graph: True, gen throughput (token/s): 152.00, #queue-req: 0, 


[2025-08-22 15:30:05] Decode batch. #running-req: 1, #token: 854, token usage: 0.04, cuda graph: True, gen throughput (token/s): 162.37, #queue-req: 0, 
[2025-08-22 15:30:05] INFO:     127.0.0.1:45220 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Native API and SGLang Runtime (SRT)

### JSON

**Using Pydantic**

In [7]:
import requests
from pydantic import BaseModel, Field
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-7B")


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


messages = [
    {
        "role": "assistant",
        "content": "Give me the information and population of the capital of France in the JSON format.",
    },
]
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
# Make API request
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json.dumps(CapitalInfo.model_json_schema()),
        },
    },
)
print(response.json())


reasoing_content = response.json()["text"].split("</think>")[0]
content = response.json()["text"].split("</think>")[1]
print_highlight(f"reasoing_content: {reasoing_content}\n\ncontent: {content}")

[2025-08-22 15:30:05] Prefill batch. #new-seq: 1, #new-token: 22, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 15:30:06] Decode batch. #running-req: 1, #token: 50, token usage: 0.00, cuda graph: True, gen throughput (token/s): 45.53, #queue-req: 0, 


[2025-08-22 15:30:06] Decode batch. #running-req: 1, #token: 90, token usage: 0.00, cuda graph: True, gen throughput (token/s): 167.59, #queue-req: 0, 


[2025-08-22 15:30:06] Decode batch. #running-req: 1, #token: 130, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.54, #queue-req: 0, 


[2025-08-22 15:30:06] Decode batch. #running-req: 1, #token: 170, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.75, #queue-req: 0, 


[2025-08-22 15:30:06] Decode batch. #running-req: 1, #token: 210, token usage: 0.01, cuda graph: True, gen throughput (token/s): 158.41, #queue-req: 0, 


[2025-08-22 15:30:07] Decode batch. #running-req: 1, #token: 250, token usage: 0.01, cuda graph: True, gen throughput (token/s): 168.30, #queue-req: 0, 


[2025-08-22 15:30:07] Decode batch. #running-req: 1, #token: 290, token usage: 0.01, cuda graph: True, gen throughput (token/s): 169.08, #queue-req: 0, 


[2025-08-22 15:30:07] Decode batch. #running-req: 1, #token: 330, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.99, #queue-req: 0, 


[2025-08-22 15:30:07] Decode batch. #running-req: 1, #token: 370, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.39, #queue-req: 0, 


[2025-08-22 15:30:08] Decode batch. #running-req: 1, #token: 410, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.33, #queue-req: 0, 
[2025-08-22 15:30:08] INFO:     127.0.0.1:48600 - "POST /generate HTTP/1.1" 200 OK
{'text': 'Okay, so the user is asking for the information and population of the capital of France in JSON format. Let me break this down. First, I need to identify what the capital of France is. I know that Paris is the capital, so that\'s straightforward. \n\nNext, I need to find the population. I remember that Paris is a major city, so its population is quite large. I think it\'s over 3 million, but I\'m not exactly sure of the exact number. Maybe I should double-check that. \n\nWait, I recall that the population figure can vary depending on the source and the year. The user didn\'t specify a particular year, so I should probably go with the most recent estimate. I believe the population is around 3,500,000 as of 2023. \n\nNow, I need to structure this

**JSON Schema Directly**

In [8]:
json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

# JSON
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": text,
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "json_schema": json_schema,
        },
    },
)

print_highlight(response.json())

[2025-08-22 15:30:08] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:30:08] Decode batch. #running-req: 1, #token: 45, token usage: 0.00, cuda graph: True, gen throughput (token/s): 153.99, #queue-req: 0, 


[2025-08-22 15:30:08] Decode batch. #running-req: 1, #token: 85, token usage: 0.00, cuda graph: True, gen throughput (token/s): 167.63, #queue-req: 0, 


[2025-08-22 15:30:08] Decode batch. #running-req: 1, #token: 125, token usage: 0.01, cuda graph: True, gen throughput (token/s): 167.56, #queue-req: 0, 


[2025-08-22 15:30:09] Decode batch. #running-req: 1, #token: 165, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.95, #queue-req: 0, 


...

[2025-08-22 15:30:09] Decode batch. #running-req: 1, #token: 245, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.65, #queue-req: 0, 


[2025-08-22 15:30:09] Decode batch. #running-req: 1, #token: 285, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.59, #queue-req: 0, 


[2025-08-22 15:30:10] Decode batch. #running-req: 1, #token: 325, token usage: 0.02, cuda graph: True, gen throughput (token/s): 161.02, #queue-req: 0, 


[2025-08-22 15:30:10] Decode batch. #running-req: 1, #token: 365, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.48, #queue-req: 0, 


[2025-08-22 15:30:10] Decode batch. #running-req: 1, #token: 405, token usage: 0.02, cuda graph: True, gen throughput (token/s): 165.37, #queue-req: 0, 
[2025-08-22 15:30:10] INFO:     127.0.0.1:48602 - "POST /generate HTTP/1.1" 200 OK


### EBNF

In [9]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Give me the information of the capital of France.",
        "sampling_params": {
            "max_new_tokens": 2048,
            "temperature": 0,
            "n": 3,
            "ebnf": (
                "root ::= city | description\n"
                'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
                'description ::= city " is " status\n'
                'status ::= "the capital of " country\n'
                'country ::= "England" | "France" | "Germany" | "Italy"'
            ),
        },
        "stream": False,
        "return_logprob": False,
    },
)

print(response.json())

[2025-08-22 15:30:10] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 15:30:10] Prefill batch. #new-seq: 2, #new-token: 2, #cached-token: 20, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 15:30:10] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 10, token usage: 0.00, #running-req: 2, #queue-req: 0, 


[2025-08-22 15:30:10] Decode batch. #running-req: 3, #token: 119, token usage: 0.01, cuda graph: True, gen throughput (token/s): 306.69, #queue-req: 0, 


[2025-08-22 15:30:11] Decode batch. #running-req: 3, #token: 239, token usage: 0.01, cuda graph: True, gen throughput (token/s): 489.13, #queue-req: 0, 


[2025-08-22 15:30:11] Decode batch. #running-req: 3, #token: 359, token usage: 0.02, cuda graph: True, gen throughput (token/s): 476.99, #queue-req: 0, 


[2025-08-22 15:30:11] Decode batch. #running-req: 3, #token: 479, token usage: 0.02, cuda graph: True, gen throughput (token/s): 483.35, #queue-req: 0, 


[2025-08-22 15:30:11] Decode batch. #running-req: 1, #token: 207, token usage: 0.01, cuda graph: True, gen throughput (token/s): 427.37, #queue-req: 0, 


[2025-08-22 15:30:12] Decode batch. #running-req: 1, #token: 247, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.76, #queue-req: 0, 
[2025-08-22 15:30:12] INFO:     127.0.0.1:48608 - "POST /generate HTTP/1.1" 200 OK
[{'text': "\nThe capital of France is Paris.\n\nThe user has provided the correct information that the capital of France is Paris. However, the user's request was to provide the information of the capital of France, which is a bit ambiguous. It could be interpreted as wanting more detailed information about Paris, such as its history, culture, landmarks, or government. Alternatively, it might mean they want to confirm that Paris is indeed the capital, which they already know. \n\nTo better assist the user, it would be helpful to ask for clarification. If they want more details about Paris, I can provide a comprehensive overview. If they just need confirmation, then the current response suffices. \n\nAdditionally, I can offer to provide more information o

### Regular expression

In [10]:
response = requests.post(
    f"http://localhost:{port}/generate",
    json={
        "text": "Paris is the capital of",
        "sampling_params": {
            "temperature": 0,
            "max_new_tokens": 2048,
            "regex": "(France|England)",
        },
    },
)
print(response.json())

[2025-08-22 15:30:12] Prefill batch. #new-seq: 1, #new-token: 5, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:30:12] Decode batch. #running-req: 1, #token: 28, token usage: 0.00, cuda graph: True, gen throughput (token/s): 154.46, #queue-req: 0, 


[2025-08-22 15:30:12] Decode batch. #running-req: 1, #token: 68, token usage: 0.00, cuda graph: True, gen throughput (token/s): 167.63, #queue-req: 0, 


[2025-08-22 15:30:12] Decode batch. #running-req: 1, #token: 108, token usage: 0.01, cuda graph: True, gen throughput (token/s): 170.00, #queue-req: 0, 


[2025-08-22 15:30:13] Decode batch. #running-req: 1, #token: 148, token usage: 0.01, cuda graph: True, gen throughput (token/s): 170.24, #queue-req: 0, 


...

[2025-08-22 15:30:23] Decode batch. #running-req: 1, #token: 1868, token usage: 0.09, cuda graph: True, gen throughput (token/s): 165.93, #queue-req: 0, 


[2025-08-22 15:30:24] Decode batch. #running-req: 1, #token: 1908, token usage: 0.09, cuda graph: True, gen throughput (token/s): 167.99, #queue-req: 0, 


[2025-08-22 15:30:24] Decode batch. #running-req: 1, #token: 1948, token usage: 0.10, cuda graph: True, gen throughput (token/s): 167.80, #queue-req: 0, 


[2025-08-22 15:30:24] Decode batch. #running-req: 1, #token: 1988, token usage: 0.10, cuda graph: True, gen throughput (token/s): 164.70, #queue-req: 0, 


[2025-08-22 15:30:24] Decode batch. #running-req: 1, #token: 2028, token usage: 0.10, cuda graph: True, gen throughput (token/s): 164.68, #queue-req: 0, 
[2025-08-22 15:30:25] INFO:     127.0.0.1:48618 - "POST /generate HTTP/1.1" 200 OK
{'text': ' France, and the \n\\( n \\)  \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\( m \\) \\( k \\) \\( l \\) \\(

### Structural Tag

In [11]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
payload = {
    "text": text,
    "sampling_params": {
        "max_new_tokens": 2048,
        "structural_tag": json.dumps(
            {
                "type": "structural_tag",
                "structures": [
                    {
                        "begin": "<function=get_current_weather>",
                        "schema": schema_get_current_weather,
                        "end": "</function>",
                    },
                    {
                        "begin": "<function=get_current_date>",
                        "schema": schema_get_current_date,
                        "end": "</function>",
                    },
                ],
                "triggers": ["<function="],
            }
        ),
    },
}


# Send POST request to the API endpoint
response = requests.post(f"http://localhost:{port}/generate", json=payload)
print_highlight(response.json())

[2025-08-22 15:30:25] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 22, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 15:30:25] Decode batch. #running-req: 1, #token: 37, token usage: 0.00, cuda graph: True, gen throughput (token/s): 150.47, #queue-req: 0, 


[2025-08-22 15:30:25] Decode batch. #running-req: 1, #token: 77, token usage: 0.00, cuda graph: True, gen throughput (token/s): 165.21, #queue-req: 0, 


[2025-08-22 15:30:25] Decode batch. #running-req: 1, #token: 117, token usage: 0.01, cuda graph: True, gen throughput (token/s): 165.17, #queue-req: 0, 


[2025-08-22 15:30:25] Decode batch. #running-req: 1, #token: 157, token usage: 0.01, cuda graph: True, gen throughput (token/s): 164.10, #queue-req: 0, 


...

[2025-08-22 15:30:27] Decode batch. #running-req: 1, #token: 397, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.18, #queue-req: 0, 


[2025-08-22 15:30:27] Decode batch. #running-req: 1, #token: 437, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.00, #queue-req: 0, 


[2025-08-22 15:30:27] Decode batch. #running-req: 1, #token: 477, token usage: 0.02, cuda graph: True, gen throughput (token/s): 163.12, #queue-req: 0, 


[2025-08-22 15:30:28] Decode batch. #running-req: 1, #token: 517, token usage: 0.03, cuda graph: True, gen throughput (token/s): 162.97, #queue-req: 0, 


[2025-08-22 15:30:28] INFO:     127.0.0.1:44372 - "POST /generate HTTP/1.1" 200 OK


In [12]:
terminate_process(server_process)

## Offline Engine API

In [13]:
import sglang as sgl

llm = sgl.Engine(
    model_path="deepseek-ai/DeepSeek-R1-Distill-Qwen-7B",
    reasoning_parser="deepseek-r1",
    grammar_backend="xgrammar",
)

W0822 15:30:30.176000 4045866 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 15:30:30.176000 4045866 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


W0822 15:30:40.798000 4050340 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 15:30:40.798000 4050340 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0
[Gloo] Rank 0 is connected to 0 peer ranks. Expected number of connected peer ranks is : 0


Loading safetensors checkpoint shards:   0% Completed | 0/2 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  50% Completed | 1/2 [00:01<00:01,  1.37s/it]


Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.26s/it]
Loading safetensors checkpoint shards: 100% Completed | 2/2 [00:02<00:00,  1.28s/it]



Capturing batches (bs=4 avail_mem=9.93 GB):   0%|          | 0/3 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=9.83 GB): 100%|██████████| 3/3 [00:00<00:00, 11.01it/s]


### JSON

**Using Pydantic**

In [14]:
import json
from pydantic import BaseModel, Field


prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]


# Define the schema using Pydantic
class CapitalInfo(BaseModel):
    name: str = Field(..., pattern=r"^\w+$", description="Name of the capital city")
    population: int = Field(..., description="Population of the capital city")


sampling_params = {
    "temperature": 0,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "json_schema": json.dumps(CapitalInfo.model_json_schema()),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

**JSON Schema Directly**

In [15]:
prompts = [
    "Give me the information of the capital of China in the JSON format.",
    "Give me the information of the capital of France in the JSON format.",
    "Give me the information of the capital of Ireland in the JSON format.",
]

json_schema = json.dumps(
    {
        "type": "object",
        "properties": {
            "name": {"type": "string", "pattern": "^[\\w]+$"},
            "population": {"type": "integer"},
        },
        "required": ["name", "population"],
    }
)

sampling_params = {"temperature": 0, "max_new_tokens": 2048, "json_schema": json_schema}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of China in the JSON format.
Generated text: 
Sure! Here's the information about the capital of China, Beijing, in JSON format:

```json
{
  "name": "Beijing",
  "capital": "Yes",
  "population": "Over 30 million",
  "founded": "1248",
  "Nickname": "The Heaven on Earth",
  "Location": "Northern China",
  "OfficialLanguages": [
    "Mandarin Chinese",
    "Bingyuan Chinese",
    "Tibetan",
    "Hui",
    "Mongolian",
    "Yugou",
    "Tibetan",
    "Hui",
    "Mongolian"
  ],
  "KeySights": [
    "The Great Wall of China",
    "Forbidden City",
    "Tiananmen Square",
    "Beijing Museum",
    "Yuanmingyuan"
  ],
  "Climate": "Temperate"
}
```

Let me know if you need anything else!
Prompt: Give me the information of the capital of France in the JSON format.
Generated text: 
Sure! Here's the information about the capital of France, Paris, in JSON format:

```json
{
  "name": "Paris",
  "country": "France",
  "coordinates": {
    "latitude"

### EBNF


In [16]:
prompts = [
    "Give me the information of the capital of France.",
    "Give me the information of the capital of Germany.",
    "Give me the information of the capital of Italy.",
]

sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "ebnf": (
        "root ::= city | description\n"
        'city ::= "London" | "Paris" | "Berlin" | "Rome"\n'
        'description ::= city " is " status\n'
        'status ::= "the capital of " country\n'
        'country ::= "England" | "France" | "Germany" | "Italy"'
    ),
}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Give me the information of the capital of France.
Generated text: 
The capital of France is Paris. But wait, isn't it also the administrative capital?

Yes, Paris is both the political and administrative capital of France.

Okay, so I need to provide information about the capital of France, specifically focusing on its status as both a political and administrative center.

Alright, so the user is asking about the capital of France. They mentioned that Paris is the capital, but also clarified that it's both the political and administrative center. So I need to make sure to highlight both aspects.

I should structure the response clearly. Maybe start with the name, then explain its role as the political center, then its
Prompt: Give me the information of the capital of Germany.
Generated text: 
Alright, so I need to figure out the information about the capital of Germany. Hmm, let's see. I remember that the capital of a country is usually its most important city, right? Germany i

### Regular expression

In [17]:
prompts = [
    "Please provide information about London as a major global city:",
    "Please provide information about Paris as a major global city:",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95, "regex": "(France|England)"}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Please provide information about London as a major global city:
Generated text:  its location, economic status, cultural significance, and the role of the City of London.

London is one of the most important cities in the world, known for its status as the capital of the United Kingdom. It is located in western Europe, along the River Thames. The city is a global financial, cultural, and transportation hub. London's economic status is largely driven by its financial services industry, which employs a significant portion of its workforce. The City of London, also known as the Financial District, is the heart of this industry, home to major banks, investment banks, insurance companies, and stock exchanges. This area has been a
Prompt: Please provide information about Paris as a major global city:
Generated text:  its location, cultural significance, and economic strength.

9-10 sentences.
Alright, I need to provide information about Paris as a major global city. The user specifie

In [18]:
text = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
prompts = [text]


sampling_params = {
    "temperature": 0.8,
    "top_p": 0.95,
    "max_new_tokens": 2048,
    "structural_tag": json.dumps(
        {
            "type": "structural_tag",
            "structures": [
                {
                    "begin": "<function=get_current_weather>",
                    "schema": schema_get_current_weather,
                    "end": "</function>",
                },
                {
                    "begin": "<function=get_current_date>",
                    "schema": schema_get_current_date,
                    "end": "</function>",
                },
            ],
            "triggers": ["<function="],
        }
    ),
}


# Send POST request to the API endpoint
outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: <｜begin▁of▁sentence｜><｜Assistant｜>Give me the information and population of the capital of France in the JSON format.<｜end▁of▁sentence｜><｜Assistant｜><think>

Generated text: Okay, so I need to figure out the capital of France and its population. Hmm, I remember that the capital of France is Paris. I'm pretty sure that's correct, but maybe I should double-check. Yeah, Paris is definitely the capital.

Now, about the population. I think it's a big city, so the population should be in the millions. I've heard it's around 20-something million. Let me see, I think it was about 21.5 million. Wait, is that accurate? I'm not entirely sure. I should probably confirm the exact number. Maybe the latest data says 2022, but I think it's updated regularly. So, 21.5 million seems about right as of recent estimates.

I should present this information in JSON format. So, I'll structure it with a key "capital" for the name and "population" for the number. I need to make sure the population is re

In [19]:
llm.shutdown()